In [1]:
import pandas as pd
import json
import ast
import numpy as np
import datetime

### 1. Conectarse al enlace

In [2]:
#Read the data as json file from the url given
df = pd.read_json('https://api.stackexchange.com/2.2/search?order=desc&sort=activity&intitle=perl&site=stackoverflow')

In [3]:
df.head() 

,items,has_more,quota_max,quota_remaining
0,"{'tags': ['perl', 'tcsh'], 'owner': {'reputati...",True,300,294
1,"{'tags': ['perl', 'utf-8'], 'owner': {'reputat...",True,300,294
2,"{'tags': ['perl'], 'owner': {'reputation': 756...",True,300,294
3,"{'tags': ['perl', 'unix', 'ldap'], 'owner': {'...",True,300,294
4,"{'tags': ['perl'], 'owner': {'reputation': 1, ...",True,300,294


As shown above, we can observer that the actually data is nested in the column "items" of our pandas dataframe.
We need to extract this data as is the information that we actually need.

In [4]:
#In a new dataframe I gonna extract the json files from the "items" columns with the method json_normalize
data = pd.DataFrame()
data = pd.io.json.json_normalize(df['items'])

C:\Users\Gerardo\AppData\Local\Temp/ipykernel_159344/573332716.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  data = pd.io.json.json_normalize(df['items'])


In [5]:
#The dataframe extracted 
data.head()

,tags,is_answered,view_count,answer_count,score,last_activity_date,creation_date,last_edit_date,question_id,content_license,...,owner.reputation,owner.user_id,owner.user_type,owner.profile_image,owner.display_name,owner.link,accepted_answer_id,closed_date,closed_reason,owner.accept_rate
0,"[perl, tcsh]",False,46,2,0,1660339816,1660321435,1.660323e+09,73337124,CC BY-SA 4.0,...,21.0,2514550.0,registered,https://www.gravatar.com/avatar/9416c716f2d690...,Nibblerglozer,https://stackoverflow.com/users/2514550/nibble...,NaN,NaN,NaN,NaN
1,"[perl, utf-8]",False,28,2,0,1660339268,1660322411,NaN,73337329,CC BY-SA 4.0,...,31.0,16179803.0,registered,https://www.gravatar.com/avatar/25208ed61cac61...,amir,https://stackoverflow.com/users/16179803/amir,NaN,NaN,NaN,NaN
2,[perl],True,55,1,2,1660335430,1660317970,1.660335e+09,73336460,CC BY-SA 4.0,...,7569.0,3387716.0,registered,https://www.gravatar.com/avatar/bcad6c8d7a64b0...,Fravadona,https://stackoverflow.com/users/3387716/fravadona,NaN,NaN,NaN,NaN
3,"[perl, unix, ldap]",True,236,1,0,1660331406,1558539974,1.558548e+09,56260614,CC BY-SA 4.0,...,35.0,8060165.0,registered,https://lh6.googleusercontent.com/-mdywERHBtWc...,Manu Gal&#225;n,https://stackoverflow.com/users/8060165/manu-g...,56261593.0,NaN,NaN,NaN
4,[perl],True,37,2,-1,1660323151,1660315012,NaN,73335798,NaN,...,1.0,19751579.0,registered,https://lh3.googleusercontent.com/a-/AFdZucqIV...,Abhimanyu Gupta,https://stackoverflow.com/users/19751579/abhim...,NaN,1.660323e+09,Not suitable for this site,NaN


In [6]:
data.columns

Index(['tags', 'is_answered', 'view_count', 'answer_count', 'score',
       'last_activity_date', 'creation_date', 'last_edit_date', 'question_id',
       'content_license', 'link', 'title', 'owner.reputation', 'owner.user_id',
       'owner.user_type', 'owner.profile_image', 'owner.display_name',
       'owner.link', 'accepted_answer_id', 'closed_date', 'closed_reason',
       'owner.accept_rate'],
      dtype='object')

### 2. Obtener el numero de respuestas contestadas y no contestadas

In [7]:
#We count the values from the "is_answered" columns 
data.is_answered.value_counts()

True     23
False     7
Name: is_answered, dtype: int64

### 3. Obtener la respuesta con menor numero de vistas

In [8]:
less_view = data["view_count"].min()

data.loc[data.view_count == less_view ]

,tags,is_answered,view_count,answer_count,score,last_activity_date,creation_date,last_edit_date,question_id,content_license,...,owner.reputation,owner.user_id,owner.user_type,owner.profile_image,owner.display_name,owner.link,accepted_answer_id,closed_date,closed_reason,owner.accept_rate
15,"[perl, conda, slurm]",False,26,0,1,1660045700,1659957476,1.660046e+09,73277012,NaN,...,23.0,17665505.0,registered,https://www.gravatar.com/avatar/a7702facff15bc...,btredcup,https://stackoverflow.com/users/17665505/btredcup,NaN,1.660046e+09,Duplicate,NaN


### 4. Obtener la respuesta mas vieja y mas actual

In [16]:
data['creation_date'] = pd.to_datetime(data['creation_date'],unit='s')

In [17]:
data

,tags,is_answered,view_count,answer_count,score,last_activity_date,creation_date,last_edit_date,question_id,content_license,...,owner.reputation,owner.user_id,owner.user_type,owner.profile_image,owner.display_name,owner.link,accepted_answer_id,closed_date,closed_reason,owner.accept_rate
0,"[perl, tcsh]",False,46,2,0,1660339816,2022-08-12 16:23:55,1.660323e+09,73337124,CC BY-SA 4.0,...,21.0,2514550.0,registered,https://www.gravatar.com/avatar/9416c716f2d690...,Nibblerglozer,https://stackoverflow.com/users/2514550/nibble...,NaN,NaN,NaN,NaN
1,"[perl, utf-8]",False,28,2,0,1660339268,2022-08-12 16:40:11,NaN,73337329,CC BY-SA 4.0,...,31.0,16179803.0,registered,https://www.gravatar.com/avatar/25208ed61cac61...,amir,https://stackoverflow.com/users/16179803/amir,NaN,NaN,NaN,NaN
2,[perl],True,55,1,2,1660335430,2022-08-12 15:26:10,1.660335e+09,73336460,CC BY-SA 4.0,...,7569.0,3387716.0,registered,https://www.gravatar.com/avatar/bcad6c8d7a64b0...,Fravadona,https://stackoverflow.com/users/3387716/fravadona,NaN,NaN,NaN,NaN
3,"[perl, unix, ldap]",True,236,1,0,1660331406,2019-05-22 15:46:14,1.558548e+09,56260614,CC BY-SA 4.0,...,35.0,8060165.0,registered,https://lh6.googleusercontent.com/-mdywERHBtWc...,Manu Gal&#225;n,https://stackoverflow.com/users/8060165/manu-g...,56261593.0,NaN,NaN,NaN
4,[perl],True,37,2,-1,1660323151,2022-08-12 14:36:52,NaN,73335798,NaN,...,1.0,19751579.0,registered,https://lh3.googleusercontent.com/a-/AFdZucqIV...,Abhimanyu Gupta,https://stackoverflow.com/users/19751579/abhim...,NaN,1.660323e+09,Not suitable for this site,NaN
5,"[perl, cross-platform]",True,69322,11,60,1660317247,2008-12-02 16:59:00,1.228354e+09,334686,CC BY-SA 2.5,...,25631.0,6444.0,registered,https://www.gravatar.com/avatar/1510549f9353c9...,mxcl,https://stackoverflow.com/users/6444/mxcl,334700.0,NaN,NaN,78.0
6,"[linux, perl, debian]",False,37,0,-2,1660260670,2022-08-11 23:31:10,NaN,73327847,NaN,...,17.0,11488241.0,registered,https://i.stack.imgur.com/hpe5o.jpg?s=256&g=1,Ahmed Asakra,https://stackoverflow.com/users/11488241/ahmed...,NaN,1.660269e+09,Needs details or clarity,NaN
7,[zsh],True,46,3,1,1660254064,2022-08-09 17:19:11,1.660076e+09,73295790,CC BY-SA 4.0,...,164.0,2194258.0,registered,https://i.stack.imgur.com/QHQsu.png?s=256&g=1,GBelanger,https://stackoverflow.com/users/2194258/gbelanger,NaN,NaN,NaN,NaN
8,"[windows, perl]",True,71,1,2,1660134590,2022-08-08 14:47:14,1.660046e+09,73279792,CC BY-SA 4.0,...,498.0,11365539.0,registered,https://lh6.googleusercontent.com/-B5YB6Qt0o1U...,Warpstar22,https://stackoverflow.com/users/11365539/warps...,73306362.0,NaN,NaN,NaN
9,"[regex, google-bigquery]",True,31,1,0,1660131016,2022-08-10 10:46:06,1.660129e+09,73304896,CC BY-SA 4.0,...,11.0,18598417.0,registered,https://www.gravatar.com/avatar/0f516e637ff295...,Amethyst,https://stackoverflow.com/users/18598417/amethyst,73305007.0,NaN,NaN,NaN


In [18]:
max_date, min_date = data['creation_date'].max(), data['creation_date'].min()

In [19]:
#Newest ansewer
data.loc[data.creation_date==max_date]

,tags,is_answered,view_count,answer_count,score,last_activity_date,creation_date,last_edit_date,question_id,content_license,...,owner.reputation,owner.user_id,owner.user_type,owner.profile_image,owner.display_name,owner.link,accepted_answer_id,closed_date,closed_reason,owner.accept_rate
1,"[perl, utf-8]",False,28,2,0,1660339268,2022-08-12 16:40:11,NaN,73337329,CC BY-SA 4.0,...,31.0,16179803.0,registered,https://www.gravatar.com/avatar/25208ed61cac61...,amir,https://stackoverflow.com/users/16179803/amir,NaN,NaN,NaN,NaN


In [20]:
#Oldest answer
data.loc[data.creation_date==min_date]

,tags,is_answered,view_count,answer_count,score,last_activity_date,creation_date,last_edit_date,question_id,content_license,...,owner.reputation,owner.user_id,owner.user_type,owner.profile_image,owner.display_name,owner.link,accepted_answer_id,closed_date,closed_reason,owner.accept_rate
5,"[perl, cross-platform]",True,69322,11,60,1660317247,2008-12-02 16:59:00,1.228354e+09,334686,CC BY-SA 2.5,...,25631.0,6444.0,registered,https://www.gravatar.com/avatar/1510549f9353c9...,mxcl,https://stackoverflow.com/users/6444/mxcl,334700.0,NaN,NaN,78.0


### 5. Obtener la respuesta del owner que tenga una mayor reputación

In [13]:
best_reputation = data["owner.reputation"].max()
data.loc[data['owner.reputation'] == best_reputation]


,tags,is_answered,view_count,answer_count,score,last_activity_date,creation_date,last_edit_date,question_id,content_license,...,owner.reputation,owner.user_id,owner.user_type,owner.profile_image,owner.display_name,owner.link,accepted_answer_id,closed_date,closed_reason,owner.accept_rate
5,"[perl, cross-platform]",True,69322,11,60,1660317247,1228237140,1.228354e+09,334686,CC BY-SA 2.5,...,25631.0,6444.0,registered,https://www.gravatar.com/avatar/1510549f9353c9...,mxcl,https://stackoverflow.com/users/6444/mxcl,334700.0,NaN,NaN,78.0


### 6. Imprimir en consola del punto 2 al 5

In [14]:
data.iloc[2:6]

,tags,is_answered,view_count,answer_count,score,last_activity_date,creation_date,last_edit_date,question_id,content_license,...,owner.reputation,owner.user_id,owner.user_type,owner.profile_image,owner.display_name,owner.link,accepted_answer_id,closed_date,closed_reason,owner.accept_rate
2,[perl],True,55,1,2,1660335430,1660317970,1.660335e+09,73336460,CC BY-SA 4.0,...,7569.0,3387716.0,registered,https://www.gravatar.com/avatar/bcad6c8d7a64b0...,Fravadona,https://stackoverflow.com/users/3387716/fravadona,NaN,NaN,NaN,NaN
3,"[perl, unix, ldap]",True,236,1,0,1660331406,1558539974,1.558548e+09,56260614,CC BY-SA 4.0,...,35.0,8060165.0,registered,https://lh6.googleusercontent.com/-mdywERHBtWc...,Manu Gal&#225;n,https://stackoverflow.com/users/8060165/manu-g...,56261593.0,NaN,NaN,NaN
4,[perl],True,37,2,-1,1660323151,1660315012,NaN,73335798,NaN,...,1.0,19751579.0,registered,https://lh3.googleusercontent.com/a-/AFdZucqIV...,Abhimanyu Gupta,https://stackoverflow.com/users/19751579/abhim...,NaN,1.660323e+09,Not suitable for this site,NaN
5,"[perl, cross-platform]",True,69322,11,60,1660317247,1228237140,1.228354e+09,334686,CC BY-SA 2.5,...,25631.0,6444.0,registered,https://www.gravatar.com/avatar/1510549f9353c9...,mxcl,https://stackoverflow.com/users/6444/mxcl,334700.0,NaN,NaN,78.0
